Split a sound into its component sine waves. Output audio, adding successive sine waves one at a time.

Used in this post: http://www.tremblingsandwarblings.com/2017/05/fourier-transform-spectrum/

And this video: https://www.youtube.com/watch?v=BvUMfnQucP4

In [ ]:
# Use Python 2.7
# Uses sms-tools-enhanced package (see readme)

In [1]:
import sys
import os
sys.path.append('../sms-tools-enhanced/software/models/')
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import get_window
import utilFunctions as UF
import sineModel as SM
import harmonicModel as HM
import hprModel as HPR

In [2]:
# Set input file

#inputFile='sounds/violin_G5_1_pianissimo_arco-normal.wav'
#inputFile='input_sounds/erhu6_novib.wav'
#inputFile='input_sounds/violin_B4_05_mezzo-piano_arco-normal.wav'
#inputFile='input_sounds/Erhu_B4.wav'
inputFile='sounds/Erhu_G5.wav'

In [3]:
# Set parameters

window='blackman'
M=801             # Window size
N=1024            # FFT size (power of 2, larger than M)
t=-90             # Threshold below which to discount peaks
minSineDur=0.07   # Minimum duration of sine track, else delete track
nH=50             # Number of harmonics to do STFT (Residual is what's left)
nH_s = 15         # Number of harmonics to synthesise, adding one after another
minf0=300         # For finding f0, set minimum possible value
maxf0=800         # For finding f0, set maximum possible value
f0et=7            # Allowed error when finding f0
harmDevSlope=0.01 # Allowed deviation from harmonicity in harmonics
Ns = 1024         # size of fft used in synthesis
H = 256           # hop size (has to be 1/4 of Ns)

In [4]:
# read input sound
(fs, x) = UF.wavread(inputFile)

# compute analysis window
w = get_window(window, M)

# Compute HPR model
hfreq, hmag, hphase, xr = HPR.hprModelAnal(x, fs, w, N, H, t, minSineDur, nH, minf0, maxf0, f0et, harmDevSlope)

In [5]:
# Synthesise the sound while adding in successive harmonics

soundarr = np.array([])                  # Container to store sounds as we add in successive harmonics
y = np.array([])                         # The output sound, at each addition of harmonics
j_ind = 0
for j in range(0,15):                    # Add in successive harmonics
    # synthesize the harmonics
    y = SM.sineModelSynth_uptoj(hfreq, hmag, hphase, Ns, H, fs, j)
    
    soundarr = np.append(soundarr, y)
    j_ind = j
    
# First nH_s harmonics plus residual (non-harmonic) part
harm_plus_res = y[:min(y.size, xr.size)] + xr[:min(y.size, xr.size)]

In [6]:
# Write output files

# output sound file (monophonic with sampling rate of 44100)
outputFileSuc = 'output_sounds/' + os.path.basename(inputFile)[:-4] + '_harmonicSeries' + '.wav'
outputFileHarmRes = 'output_sounds/' + os.path.basename(inputFile)[:-4] + '_harm+res' + '.wav'
outputFileLast = 'output_sounds/' + os.path.basename(inputFile)[:-4] + '_harm_' + str(j_ind) + '.wav'
outputFileRes = 'output_sounds/' + os.path.basename(inputFile)[:-4] + '_res' + '.wav'

# write the sound resulting from harmonic analysis
UF.wavwrite(soundarr, fs, outputFileSuc)                # Adding first nH_s harmonics in succession
UF.wavwrite(harm_plus_res, fs, outputFileHarmRes)       # Sum of first nH_s harmonics plus residual (non-harmonic) part
UF.wavwrite(y, fs, outputFileLast)                      # The last (nH_s-th) harmonic
UF.wavwrite(xr, fs, outputFileRes)                      # Residual (non-harmonic) part